In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import seaborn as sns
import tqdm.notebook as tq


In [15]:
df = pd.read_csv("train_df_renamed.csv")

In [17]:
df['Household level identifier'].nunique()

2685

In [4]:
df = df.drop(columns=['Id'])

In [278]:
heads = df.loc[df['=1 if household head']==1].copy()

In [5]:
cb = pd.read_csv("/Users/yingcheng/Desktop/SUTD dsci/Project/CZ4041_Project/codebook.csv")

In [10]:
cb.head()

,Variable name,Variable description
0,v2a1,Monthly rent payment
1,hacdor,=1 Overcrowding by bedrooms
2,rooms,number of all rooms in the house
3,hacapo,=1 Overcrowding by rooms
4,v14a,=1 has toilet in the household


In [7]:
aggr_mean_list = list(cb['Variable name'])

In [8]:
aggr_mean_list_1 = []

# hello = cb.loc[cb['Variable name']=="v2a1","Variable description"][0]
for i in aggr_mean_list:
#     cb.query('Variable name== {}'.format(i))['Variable description']
    hello = cb[cb['Variable name'] == i]['Variable description'].values[0]
#     ag
    aggr_mean_list_1.append(hello)

In [9]:
print(aggr_mean_list_1)

['Monthly rent payment', '=1 Overcrowding by bedrooms', ' number of all rooms in the house', '=1 Overcrowding by rooms', '=1 has toilet in the household', '=1 if the household has refrigerator', 'owns a tablet', 'number of tablets household owns', 'Males younger than 12 years of age', 'Males 12 years of age and older', 'Total males in the household', 'Females younger than 12 years of age', 'Females 12 years of age and older', 'Total females in the household', 'persons younger than 12 years of age', 'persons 12 years of age and older', 'Total persons in the household', 'size of the household', 'TamViv', 'years of schooling', 'Years behind in school', 'household size', '=1 if predominant material on the outside wall is block or brick', '=1 if predominant material on the outside wall is socket (wood, zinc or absbesto', '=1 if predominant material on the outside wall is prefabricated or cement', '=1 if predominant material on the outside wall is waste material', '=1 if predominant material

In [389]:
print(aggr_mean_list_1)

['Years behind in school', '=1 if disable person', '=1 if male', '=1 if female', '=1 if less than 10 years old', '=1 if free or coupled uunion', '=1 if married', '=1 if divorced', '=1 if separated', '=1 if widow/er', '=1 if single', '=1 if spouse/partner', '=1 if son/doughter', '=1 if stepson/doughter', '=1 if son/doughter in law', '=1 if grandson/doughter', '=1 if mother/father', '=1 if father/mother in law', '=1 if brother/sister', '=1 if brother/sister in law', '=1 if other family member', '=1 if other non family member', '=1 no level of education', '=1 incomplete primary', '=1 complete primary', '=1 incomplete academic secondary level', '=1 complete academic secondary level', '=1 incomplete technical secondary level', '=1 complete technical secondary level', '=1 undergraduate and higher education', '=1 postgraduate higher education']


In [279]:
# df of only heads of household
heads

,Household level identifier,Monthly rent payment,=1 Overcrowding by bedrooms,number of all rooms in the house,=1 Overcrowding by rooms,=1 has toilet in the household,=1 if the household has refrigerator,owns a tablet,number of tablets household owns,Males younger than 12 years of age,...,Age in years,escolari squared,age squared,hogar_total squared,edjefe squared,hogar_nin squared,overcrowding squared,dependency squared,meaned squared,Age squared
2,72958b30c,0.0,0,5,0,1,1,0,0.0,1,...,41,289,1681,9,0,1,2.25,0.25,272.25000,1681
3,5b598fbc9,0.0,0,14,0,1,1,1,1.0,0,...,59,256,3481,1,256,0,1.00,0.00,256.00000,3481
4,1e2fc704e,175000.0,0,4,0,1,1,1,1.0,0,...,18,121,324,1,0,1,0.25,64.00,100.50922,324
6,8ee7365a8,400000.0,0,3,0,1,1,1,1.0,0,...,42,121,1764,4,0,0,1.00,0.00,182.25000,1764
8,ff69a6fc8,300000.0,0,6,0,1,1,1,1.0,0,...,22,196,484,4,0,0,0.25,0.00,196.00000,484
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23836,e44cb9969,0.0,0,4,0,0,0,0,0.0,1,...,68,9,4624,4,0,1,1.00,64.00,9.00000,4624
23841,935a65ffa,0.0,0,4,0,1,1,0,0.0,1,...,42,16,1764,36,16,16,9.00,4.00,25.00000,1764
23843,2edb6f51e,0.0,0,5,0,1,1,0,0.0,0,...,47,0,2209,9,0,1,1.00,0.25,30.25000,2209
23847,3aa78c56b,0.0,1,2,1,1,1,0,0.0,0,...,57,25,3249,36,25,4,36.00,0.25,33.06250,3249


In [14]:
all_equal = df.groupby('Household level identifier')['Target'].apply(lambda x: x.nunique() == 1)

KeyError: 'Column not found: Target'

In [281]:
not_equal = all_equal[all_equal != True]
print('No of households where target values are not all the same: %s'%(len(not_equal)))

No of households where target values are not all the same: 85


In [282]:
# e.g. of households with different target values
# df[df['Household level identifier'] == not_equal.index[0]][['Household level identifier', '=1 if household head', 'Target']]
df[df['Household level identifier'] == not_equal.index[0]][['Household level identifier', '=1 if household head']]

,Household level identifier,=1 if household head


In [283]:
households_with_head = df.groupby('Household level identifier')['=1 if household head'].sum()

In [284]:
households_no_heads = df.loc[df['Household level identifier'].isin(households_with_head[households_with_head == 0].index), :]

In [285]:
print('No. of households with no heads: %s' %households_no_heads['Household level identifier'].nunique())

No. of households with no heads: 18


In [286]:
# households_nh_equal = households_no_heads.groupby('Household level identifier')['Target'].apply(lambda x:x.nunique()==1)


In [287]:
# there are no households with no heads and different target labels => we don't have to worry about this possibility

print('No. of households with no heads & have different labels: %s' %sum(households_nh_equal==False))

No. of households with no heads & have different labels: 0


In [288]:
# change households' targets with different target labels to be same as that household head

# for h in not_equal.index:
#     label = int(df[(df['Household level identifier'] == h) & (df['=1 if household head'] == 1)]['Target'])
#     df.loc[df['Household level identifier'] == h, 'Target'] = label

In [289]:
# all_equal_1 = df.groupby('Household level identifier')['Target'].apply(lambda x:x.nunique()==1)


In [290]:
# confirmed that there are no longer households with different target labels
# print(len(all_equal_1[all_equal_1 != True]))

In [291]:
print(len(df))

23856


In [292]:
#find number column(s) where values are entirely made up of 1 value
cnt = 0
for x in df:
    items_counts = df[x].value_counts()
    max_item = items_counts.max()
    if max_item == 8608:
        print(x)
        cnt +=1
print(cnt)

0


In [293]:
thres = 0.99
cnt = 0
drop_ls = []

# for x in df:
#     items_counts = df[x].value_counts()
#     max_item = items_counts.max()
#     if max_item/len(df) > thres:
#         print(x)
#         drop_ls.append(x)
#         cnt +=1
# print('No. of columns where >0.99 of values are same: %s' %cnt)


In [294]:
print('no of columns initially: %s' %len(df.columns))
df.drop(drop_ls, axis=1, inplace=True)
print('no of columns after dropping columns: %s' %len(df.columns))

no of columns initially: 139
no of columns after dropping columns: 139


In [304]:
# id_ = [ 'Household level identifier', 'Target']
id_ =  [ 'Household level identifier']

ind_bool = ['owns a tablet', '=1 if disable person', '=1 if male', '=1 if female', '=1 if less than 10 years old', '=1 if free or coupled uunion', '=1 if married', '=1 if divorced', '=1 if separated', '=1 if widow/er', '=1 if single', '=1 if household head', '=1 if spouse/partner', '=1 if son/doughter', '=1 if stepson/doughter', '=1 if son/doughter in law', '=1 if grandson/doughter', '=1 if mother/father', '=1 if father/mother in law', '=1 if brother/sister', '=1 if brother/sister in law', '=1 if other family member', '=1 if other non family member', '=1 no level of education', '=1 incomplete primary', '=1 complete primary', '=1 incomplete academic secondary level', '=1 complete academic secondary level', '=1 incomplete technical secondary level', '=1 complete technical secondary level', '=1 undergraduate and higher education', '=1 postgraduate higher education', '=1 if mobile phone']
ind_ordered = ['years of schooling', 'Age in years']

hh_bool = ['=1 Overcrowding by bedrooms', '=1 Overcrowding by rooms', '=1 has toilet in the household', '=1 if the household has refrigerator', '=1 if predominant material on the outside wall is block or brick', '=1 if predominant material on the outside wall is socket (wood, zinc or absbesto', '=1 if predominant material on the outside wall is prefabricated or cement', '=1 if predominant material on the floor is cement', '=1 if predominant material on the outside wall is waste material', '=1 if predominant material on the outside wall is wood ', '=1 if predominant material on the outside wall is zink', '=1 if predominant material on the outside wall is natural fibers', '=1 if predominant material on the outside wall is other', '=1 if predominant material on the floor is mosaic, ceramic, terrazo', '=1 if predominant material on the floor is other', '=1 if predominant material on the floor is  natural material', '=1 if no floor at the household', '=1 if predominant material on the floor is wood', '=1 if predominant material on the roof is metal foil or zink', '=1 if predominant material on the roof is fiber cement, mezzanine ', '=1 if predominant material on the roof is natural fibers', '=1 if predominant material on the roof is other', '=1 if the house has ceiling', '=1 if water provision inside the dwelling', '=1 if water provision outside the dwelling', '=1 if no water provision', '=1 electricity from CNFL, ICE, ESPH/JASEC', '=1 electricity from private plant', '=1 no electricity in the dwelling', '=1 electricity from cooperative', '=1 no toilet in the dwelling', '=1 toilet connected to sewer or cesspool', '=1 toilet connected to  septic tank', '=1 toilet connected to black hole or letrine', '=1 toilet connected to other system', '=1 no main source of energy used for cooking (no kitchen)', '=1 main source of energy used for cooking electricity', '=1 main source of energy used for cooking gas', '=1 main source of energy used for cooking wood charcoal', '=1 if rubbish disposal mainly by tanker truck', '=1 if rubbish disposal mainly by botan hollow or buried', '=1 if rubbish disposal mainly by burning', '=1 if rubbish disposal mainly by throwing in an unoccupied space', '=1 if rubbish disposal mainly other', '=1 if walls are bad', '=1 if walls are regular', '=1 if walls are good', '=1 if roof are bad', '=1 if roof are regular', '=1 if roof are good', '=1 if floor are bad', '=1 if floor are regular', '=1 if floor are good', '=1 own and fully paid house', '=1 own, paying in installments', '=1 rented', '=1 precarious', '=1 other(assigned, borrowed)', '=1 if the household has notebook or desktop computer', '=1 if the household has TV', '=1 region Central', '=1 region Chorotega', '=1 region PacÃƒÂ\xadfico central', '=1 region Brunca', '=1 region Huetar AtlÃƒÂ¡ntica', '=1 region Huetar Norte', '=1 zona urbana', '=2 zona rural']
hh_ordered = [' number of all rooms in the house', 'Males younger than 12 years of age', 'Males 12 years of age and older', 'Total males in the household', 'Females younger than 12 years of age', 'Females 12 years of age and older', 'Total females in the household', 'persons younger than 12 years of age', 'persons 12 years of age and older', 'Total persons in the household', 'number of tablets household owns', 'size of the household', 'TamViv', 'household size', 'Number of children 0 to 19 in household', 'Number of adults in household', '# of individuals 65+ in the household', '# of total individuals in the household', 'number of bedrooms', '# of mobile phones']
hh_cont = ['Monthly rent payment', 'Dependency rate', 'years of education of male head of household', 'years of education of female head of household', 'average years of education for adults (18+)', '# persons per room']

sqr_values = ['escolari squared', 'age squared', 'hogar_total squared', 'edjefe squared', 'hogar_nin squared', 'overcrowding squared', 'dependency squared', 'meaned squared', 'Age squared']

drop_ls

[]

In [296]:
hh_bool = [x for x in hh_bool if x not in drop_ls]
ind_bool = [x for x in ind_bool if x not in drop_ls]
ind_ordered = [x for x in ind_ordered if x not in drop_ls]
hh_ordered = [x for x in hh_ordered if x not in drop_ls]
hh_cont = [x for x in hh_cont if x not in drop_ls]
sqr_values = [x for x in sqr_values if x not in drop_ls]

In [297]:
x = ind_bool + ind_ordered + id_ + hh_bool + hh_ordered + hh_cont + sqr_values

In [298]:
# make sure that all columns are covered

print(len(x))
print(len(df.columns))

140
139


In [299]:
ind_bool_df = df[ind_bool]
ind_ordered_df = df[ind_ordered]
hh_bool_df = df[hh_bool]
hh_ordered_df = df[hh_ordered]
hh_cont_df = df[hh_cont]
sqr_values_df = df[sqr_values]

In [300]:
for i in df:
    print(i)

Household level identifier
Monthly rent payment
=1 Overcrowding by bedrooms
 number of all rooms in the house
=1 Overcrowding by rooms
=1 has toilet in the household
=1 if the household has refrigerator
owns a tablet
number of tablets household owns
Males younger than 12 years of age
Males 12 years of age and older
Total males in the household
Females younger than 12 years of age
Females 12 years of age and older
Total females in the household
persons younger than 12 years of age
persons 12 years of age and older
Total persons in the household
size of the household
TamViv
years of schooling
household size
=1 if predominant material on the outside wall is block or brick
=1 if predominant material on the outside wall is socket (wood, zinc or absbesto
=1 if predominant material on the outside wall is prefabricated or cement
=1 if predominant material on the outside wall is waste material
=1 if predominant material on the outside wall is wood 
=1 if predominant material on the outside wall

In [301]:
#drop squared variables features

df =df.drop(columns=sqr_values)


In [302]:
print(len(df.columns))

130


## Household Level Variables

In [306]:
#subset heads of household & household level variables
heads = df.loc[df['=1 if household head'] == 1, :]
heads = heads[id_ + hh_bool + hh_cont + hh_ordered]
heads.shape

(7334, 95)

In [307]:
corr_matrix = heads.corr()

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(abs(upper[column]) > 0.95)]

In [308]:
print(to_drop)

['=1 electricity from cooperative', '=2 zona rural', 'size of the household', 'household size', '# of total individuals in the household']


In [309]:
corr_matrix.loc[corr_matrix['size of the household'].abs() > 0.9, corr_matrix['size of the household'].abs() > 0.9]

,Total persons in the household,size of the household,TamViv,household size,# of total individuals in the household
Total persons in the household,1.000000,0.998861,0.903123,0.998861,0.998861
size of the household,0.998861,1.000000,0.902313,1.000000,1.000000
TamViv,0.903123,0.902313,1.000000,0.902313,0.902313
household size,0.998861,1.000000,0.902313,1.000000,1.000000
# of total individuals in the household,0.998861,1.000000,0.902313,1.000000,1.000000


Since 'household size' has almost perfect correlation with 'size of household' and 'total individuals' in household, remove the latter 2. Remove 'total persons in household' also as it has near perfect correlation of 1 with 'household size'. 

In [310]:
heads = heads.drop(columns = ['size of the household','# of total individuals in the household','Total persons in the household'])

In [311]:
# drop '=2 zona rural' as we also have a column to indicate if house is in urban zone
heads = heads.drop(columns = '=2 zona rural')

In [390]:
aggr_mean_list_1 = []

# hello = cb.loc[cb['Variable name']=="v2a1","Variable description"][0]
for i in aggr_mean_list:
#     cb.query('Variable name== {}'.format(i))['Variable description']
    hello = cb[cb['Variable name'] == i]['Variable description'].values[0]
#     ag
    aggr_mean_list_1.append(hello)

In [315]:
# create ordinal variable electricity based on these 4 values:
# 0: No electricity
# 1: Electricity from cooperative
# 2: Electricity from CNFL, ICA, ESPH/JASEC
# 3: Electricity from private plant

electricity = []

# Assign values
for i, row in heads.iterrows():
    if row['=1 no electricity in the dwelling'] == 1:
        electricity.append(0)
    elif row['=1 electricity from cooperative'] == 1:
        electricity.append(1)
    elif row['=1 electricity from CNFL, ICE, ESPH/JASEC'] == 1:
        electricity.append(2)
    elif row['=1 electricity from private plant'] == 1:
        electricity.append(3)
    else:
        electricity.append(np.nan)
        
# Record the new variable and missing flag
heads['electricity'] = electricity
heads['electricity-missing'] = heads['electricity'].isnull()

# Remove the electricity columns
# heads = heads.drop(columns = ['noelec', 'coopele', 'public', 'planpri'])

In [316]:
#2 values with electricity count missing
heads['electricity-missing'].value_counts()

False    7323
True       11
Name: electricity-missing, dtype: int64

In [317]:
# remove old columns for electricity
heads = heads.drop(columns = ['=1 no electricity in the dwelling','=1 electricity from cooperative','=1 electricity from CNFL, ICE, ESPH/JASEC','=1 electricity from private plant'])

In [318]:
# create 'walls' categorical variable => ordinal variable as there is an obvious inherent order
# bad<regular <good
# find which column is non-zero and assign value to 'walls' column

In [319]:
heads['walls'] = np.argmax(np.array(heads[['=1 if walls are bad', '=1 if walls are regular', '=1 if walls are good']]),
                           axis = 1)

In [320]:
#drop respective variables for old walls 
heads = heads.drop(columns =['=1 if walls are bad', '=1 if walls are regular', '=1 if walls are good'])

In [321]:
# create a similar roof ordinal variable
heads['roof'] = np.argmax(np.array(heads[['=1 if roof are bad', '=1 if roof are regular', '=1 if roof are good']]),axis = 1)

In [322]:
heads = heads.drop(columns = ['=1 if roof are bad', '=1 if roof are regular', '=1 if roof are good'])

In [323]:
# create a similar floor ordinal variable
heads['floor'] = np.argmax(np.array(heads[['=1 if floor are bad', '=1 if floor are regular', '=1 if floor are good']]),axis = 1)

In [324]:
heads = heads.drop(columns = ['=1 if floor are bad', '=1 if floor are regular', '=1 if floor are good'])

In [325]:
print(len(heads.columns))

83


possible to create a new feature 'house quality' which combines the 'walls'+'floor'+'roof' qualities

In [326]:
# create feature 'no_essentials' which represents negative qualities for no toilet, no electricity, no floor, no water service and ceiling

In [327]:
heads['no_essentials'] = 1 * (heads['=1 no toilet in the dwelling'] + 
                         (heads['electricity'] == 0) + 
                         heads['=1 if no floor at the household'] + 
                         heads['=1 if water provision inside the dwelling'] + 
                         (heads['=1 if the house has ceiling'] == 0))

In [328]:
# create luxury variable which represents positive qualities (1 point for each positive variable)

In [329]:
heads['luxury'] = 1 * (heads['=1 if the household has refrigerator'] + 
                      heads['=1 if the household has notebook or desktop computer'] + 
                      (heads['number of tablets household owns'] > 0) + 
                      heads['=1 if the household has TV'])

In [330]:
# per capita variables
heads['phones-per-capita'] = heads['# of mobile phones'] / heads['TamViv']
heads['tablets-per-capita'] = heads['number of tablets household owns'] / heads['TamViv']
heads['rooms-per-capita'] = heads[' number of all rooms in the house'] / heads['TamViv']
heads['rent-per-capita'] = heads['Monthly rent payment'] / heads['TamViv']

In [331]:
hh_features = list(heads.columns)

## Individual level variables

In [332]:
ind = df[id_ + ind_bool + ind_ordered]
ind.shape

(23856, 36)

In [333]:
ind.head()

,Household level identifier,owns a tablet,=1 if disable person,=1 if male,=1 if female,=1 if less than 10 years old,=1 if free or coupled uunion,=1 if married,=1 if divorced,=1 if separated,...,=1 complete primary,=1 incomplete academic secondary level,=1 complete academic secondary level,=1 incomplete technical secondary level,=1 complete technical secondary level,=1 undergraduate and higher education,=1 postgraduate higher education,=1 if mobile phone,years of schooling,Age in years
0,72958b30c,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,4
1,72958b30c,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,1,16,41
2,72958b30c,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,1,1,17,41
3,5b598fbc9,1,0,1,0,0,0,0,0,1,...,0,0,0,0,0,1,0,1,16,59
4,1e2fc704e,1,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,1,11,18


In [334]:
corr_matrix = ind.corr()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(abs(upper[column]) > 0.95)]

to_drop

['=1 if female']

In [335]:
#remove 'male' tag since it represents same information as female tag

In [336]:
ind = ind.drop(columns = '=1 if male')

In [337]:
# instlevel1	=1 no level of education
# instlevel2	=1 incomplete primary
# instlevel3	=1 complete primary
# instlevel4	=1 incomplete academic secondary level
# instlevel5	=1 complete academic secondary level
# instlevel6	=1 incomplete technical secondary level
# instlevel7	=1 complete technical secondary level
# instlevel8	=1 undergraduate and higher education
# instlevel9	=1 postgraduate higher education


In [338]:
max_edu = ['=1 no level of education','=1 incomplete primary','=1 complete primary','=1 incomplete academic secondary level','=1 complete academic secondary level','=1 incomplete technical secondary level','=1 complete technical secondary level','=1 undergraduate and higher education','=1 postgraduate higher education']
ind[max_edu]

,=1 no level of education,=1 incomplete primary,=1 complete primary,=1 incomplete academic secondary level,=1 complete academic secondary level,=1 incomplete technical secondary level,=1 complete technical secondary level,=1 undergraduate and higher education,=1 postgraduate higher education
0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,1,0
4,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
23851,0,1,0,0,0,0,0,0,0
23852,0,0,1,0,0,0,0,0,0
23853,0,1,0,0,0,0,0,0,0
23854,0,1,0,0,0,0,0,0,0


In [339]:
# create 'max_edu' variable which records max education lvl
ind['max_edu'] = np.argmax(np.array(ind[max_edu]),axis = 1)

In [340]:
ind['max_edu']
ind = ind.drop(columns = max_edu)

In [341]:
print(len(ind.columns))

27


In [342]:
# create schooling/age variable 'schooling/age'
ind['schooling/age'] = ind['years of schooling']/ind['Age in years']

In [406]:
ind.head()

,Household level identifier,owns a tablet,=1 if disable person,=1 if female,=1 if less than 10 years old,=1 if free or coupled uunion,=1 if married,=1 if divorced,=1 if separated,=1 if widow/er,...,=1 if father/mother in law,=1 if brother/sister,=1 if brother/sister in law,=1 if other family member,=1 if other non family member,=1 if mobile phone,years of schooling,Age in years,max_edu,schooling/age
0,72958b30c,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,4,0,0.000000
1,72958b30c,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,16,41,7,0.390244
2,72958b30c,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,1,17,41,8,0.414634
3,5b598fbc9,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,16,59,7,0.271186
4,1e2fc704e,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,11,18,4,0.611111


## Aggregate individual data for each household

In [344]:
range_ = lambda x: x.max() - x.min()
range_.__name__ = 'range_'

# Group and aggregate
ind_agg = ind.groupby('Household level identifier').agg(['min', 'max', 'sum', 'count', 'std', range_])
# ind_agg.head()

In [407]:
cb[cb['Variable name'] == 'escolari']['Variable description'].values[0]

'years of schooling'

In [416]:
df_train = pd.DataFrame()
# df_test = pd.DataFrame()

aggr_mean_list = [ 'dis', 'female', 'estadocivil1', 'estadocivil2', 'estadocivil3', 'estadocivil4', 'estadocivil5', 'estadocivil6', 'estadocivil7', 'parentesco2',
             'parentesco3', 'parentesco4', 'parentesco5', 'parentesco6', 'parentesco7', 'parentesco8', 'parentesco9', 'parentesco10', 'parentesco11', 'parentesco12',
                ]

other_list = ['escolari', 'age']

aggr_mean_list_1 = []
aggr_mean_list_1.append('max_edu')
for i in aggr_mean_list:
    hello = cb[cb['Variable name'] == i]['Variable description'].values[0]
    aggr_mean_list_1.append(hello)
other_list_1 = []
for i in other_list:
    hello = cb[cb['Variable name'] == i]['Variable description'].values[0]
    other_list_1.append(hello)
# (cb['Variable name']=='escolari_age').any()



for item in aggr_mean_list_1:
    group_train_mean = ind[item].groupby(ind['Household level identifier']).mean()
#     group_test_mean = test_set[item].groupby(test_set['Household level identifier']).mean()
    new_col = item + '_aggr_mean'
    df_train[new_col] = group_train_mean
#     df_test[new_col] = group_test_mean

df_train = df_train.reset_index()

In [345]:
new_col = []
for c in ind_agg.columns.levels[0]:
    for stat in ind_agg.columns.levels[1]:
        new_col.append(f'{c}-{stat}')
        
ind_agg.columns = new_col
ind_agg.head()

,owns a tablet-min,owns a tablet-max,owns a tablet-sum,owns a tablet-count,owns a tablet-std,owns a tablet-range_,=1 if disable person-min,=1 if disable person-max,=1 if disable person-sum,=1 if disable person-count,...,max_edu-sum,max_edu-count,max_edu-std,max_edu-range_,schooling/age-min,schooling/age-max,schooling/age-sum,schooling/age-count,schooling/age-std,schooling/age-range_
Household level identifier,,,,,,,,,,,,,,,,,,,,,
000a08204,1,1,3,3,0.0,0,0,0,0,3,...,14,3,4.041452,7,0.000000,0.500000,0.900000,3,0.264575,0.500000
000bce7c4,0,0,0,2,0.0,0,0,1,1,2,...,1,2,0.707107,1,0.000000,0.083333,0.083333,2,0.058926,0.083333
001845fb0,0,0,0,4,0.0,0,0,0,0,4,...,16,4,2.160247,5,0.115385,0.526316,1.397906,4,0.183106,0.410931
003514e22,1,1,4,4,0.0,0,0,0,0,4,...,12,4,2.943920,7,0.000000,0.529412,1.095023,4,0.240643,0.529412
003b51a87,0,0,0,4,0.0,0,0,1,1,4,...,5,4,0.500000,1,0.066667,0.384615,0.822370,4,0.149839,0.317949


In [346]:
# Create correlation matrix
corr_matrix = ind_agg.corr()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(abs(upper[column]) > 0.95)]

print(f'There are {len(to_drop)} correlated columns to remove.')

There are 76 correlated columns to remove.


Remove 1 out of every pair of variables with a correlation >0.95

In [413]:
# to_drop
# 

In [349]:
ind_agg = ind_agg.drop(columns = to_drop)
ind_feats = list(ind_agg.columns)


In [360]:
final1 = pd.merge(ind_agg,heads, on = 'Household level identifier', how = 'left')

In [362]:
final1['Household level identifier'].nunique()

7352

In [363]:
final1.head()

,Household level identifier,owns a tablet-min,owns a tablet-sum,owns a tablet-count,owns a tablet-std,owns a tablet-range_,=1 if disable person-min,=1 if disable person-max,=1 if disable person-sum,=1 if female-min,...,electricity-missing,walls,roof,floor,no_essentials,luxury,phones-per-capita,tablets-per-capita,rooms-per-capita,rent-per-capita
0,000a08204,1,3,3,0.0,0,0,0,0,0,...,False,1.0,1.0,1.0,1.0,3.0,0.666667,0.333333,1.333333,55000.0
1,000bce7c4,0,0,2,0.0,0,0,1,1,0,...,False,1.0,1.0,2.0,2.0,1.0,1.500000,0.000000,2.500000,0.0
2,001845fb0,0,0,4,0.0,0,0,0,0,0,...,False,2.0,2.0,2.0,1.0,3.0,1.500000,0.000000,1.500000,75000.0
3,003514e22,1,4,4,0.0,0,0,0,0,0,...,False,2.0,1.0,0.0,1.0,3.0,1.000000,0.250000,1.250000,0.0
4,003b51a87,0,0,4,0.0,0,0,1,1,0,...,False,0.0,0.0,0.0,2.0,1.0,0.500000,0.000000,1.000000,0.0


In [350]:
# Merge on the household id
final = heads.merge(ind_agg, on = 'Household level identifier', how = 'left')

print('Final features shape: ', final.shape)

Final features shape:  (7334, 175)


In [364]:
final1.to_csv('aggregated_test_1.csv')

In [359]:
final['Household level identifier'].nunique()

7334